In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Access the OpenAI key
openai_key = os.getenv("OPENAI_API_KEY")
activeloop_key = os.getenv("ACTIVELOOP_TOKEN")


In [56]:
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.deeplake import DeepLake
from langchain.document_loaders import PyPDFLoader
import random
from langchain.memory import ConversationBufferWindowMemory
from langchain.embeddings import OpenAIEmbeddings
from tqdm import tqdm
from typing import List

from langchain.chains.openai_functions import (
    create_structured_output_chain,
)
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema import HumanMessage, SystemMessage
from pydantic import BaseModel, Field
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)

In [6]:
openai_embeddings = OpenAIEmbeddings()

In [7]:
# dataset_path = "hub://101010/text_embedding"
# # Create a DeepLake instance and add the documents
# db = DeepLake.from_documents(documents, dataset_path=dataset_path, embedding=OpenAIEmbeddings())

db = DeepLake(
    dataset_path=f"hub://siddartha10/manufacturing_CSI",  # org_id stands for your username or organization from activeloop
    embedding=openai_embeddings,
    runtime={"tensor_db": True},
    token=activeloop_key,
    # overwrite=True, # user overwrite flag if you want to overwrite the full dataset
    read_only=False,
)

Your Deep Lake dataset has been successfully created!


In [8]:
# Replace 'file_paths' with the paths to your local PDF files
file_paths = [
    "manual\'s.pdf",
    "Operation and Maintenance Manual.pdf",
    "Operations Manual.pdf"
]
# Initialize an empty list to store all pages
pages = []

# Loop through each manual and load its pages
for manual_path in file_paths:
    loader = PyPDFLoader(manual_path)
    pages1 = loader.load_and_split()
    pages.extend(pages1)

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

chunk_size = 4096
docs_new = []

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
)

for doc in pages:
    if len(doc.page_content) < chunk_size:
        docs_new.append(doc)
    else:
        docs = text_splitter.create_documents([doc.page_content])
        docs_new.extend(docs)

In [10]:
docs = db.add_documents(docs_new)

Creating 197 embeddings in 1 batches of size 197:: 100%|██████████| 1/1 [00:16<00:00, 16.52s/it]

Dataset(path='hub://siddartha10/manufacturing_CSI', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype       shape      dtype  compression
  -------    -------     -------    -------  ------- 
   text       text      (197, 1)      str     None   
 metadata     json      (197, 1)      str     None   
 embedding  embedding  (197, 1536)  float32   None   
    id        text      (197, 1)      str     None   


In [11]:
# fetch dataset docs and ids if they exist (optional you can also ingest)
docs = db.vectorstore.dataset.text.data(fetch_chunks=True, aslist=True)["value"]
ids = db.vectorstore.dataset.id.data(fetch_chunks=True, aslist=True)["value"]

In [12]:
# If we pass in a model explicitly, we need to make sure it supports the OpenAI function-calling API.
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


class Questions(BaseModel):
    """Identifying information about the manufacturing system."""

    question: str = Field(..., description="Questions about manufacturing system")


prompt_msgs = [
    SystemMessage(
        content="You are a world class expert for generating questions based on provided context. \
                You make sure the question can be answered by the text."
    ),
    HumanMessagePromptTemplate.from_template(
        "Use the given text to generate a question from the following input: {input}"
    ),
    HumanMessage(content="Tips: Make sure to answer in the correct format"),
]
prompt = ChatPromptTemplate(messages=prompt_msgs)
chain = create_structured_output_chain(Questions, llm, prompt, verbose=True)

text = """
UWM CSI Vial Filling Connected Smart Manufacturing (CSM) System The UWM CSI Vial Filling CSM  System is an intelligent manufacturing system using the latest Industry 4.0 connected advanced manufacturing equipment and techniques to produce  vials filled with varying product using a variety of filling methods and capturing process data that can be used for data analysis and system optimization.  The  Vial Filling CSM System is a platform that university faculty and students will use for both education and research to  further the advancement of a connected enterprise.   The CAM components are  integrate d seamlessly within a fully  integrated architecture and connected  enterprise using cutting -edge smart -data devices at all layers.     The process overview detailed in this section describes in general how the Vial Filling Connected Smart Manufacturing system functions as a complete system processing components.  Operational steps and slight variances in the process may differ from what is described here depending on the configuration parameters or  using the stations  in a dry cycle mode .  The Vial Filling CSM utilizes localized system configuration setting within the machine as well as process data requirements and parameters to determine how  to process the product and which stations and inspections are required to complete the production job.  When running under MES Production Center control, the process requirements and parameters are received  from the MES system as to where  and how to process and inspect the vials produced by the system .    """
questions = chain.run(input=text)
print(questions)



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a world class expert for generating questions based on provided context.                 You make sure the question can be answered by the text.
Human: Use the given text to generate a question from the following input: 
UWM CSI Vial Filling Connected Smart Manufacturing (CSM) System The UWM CSI Vial Filling CSM  System is an intelligent manufacturing system using the latest Industry 4.0 connected advanced manufacturing equipment and techniques to produce  vials filled with varying product using a variety of filling methods and capturing process data that can be used for data analysis and system optimization.  The  Vial Filling CSM System is a platform that university faculty and students will use for both education and research to  further the advancement of a connected enterprise.   The CAM components are  integrate d seamlessly within a fully  integrated architecture and connected  enterprise using cutting -

In [13]:
def generate_queries(docs: List[str], ids: List[str], n: int=100 ):
    questions = []
    relevances = []
    pbar = tqdm(total=n)
    while len(questions) < n:
        # 1. randomly draw a piece of text and relevance id
        r = random.randint(0, len(docs) - 1)
        text, label = docs[r], ids[r]

        # 2. generate queries and assign and relevance id
        generated_qs = [chain.run(input=text).question]
        questions.extend(generated_qs)
        relevances.extend([[(label, 1)] for _ in generated_qs])
        pbar.update(len(generated_qs))
        if len(questions) % 10 == 0:
            print(f"q: {len(questions)}")
    return questions[:n], relevances[:n]


chain = create_structured_output_chain(Questions, llm, prompt, verbose=False)
questions, relevances = generate_queries(docs, ids, n=500)

  2%|▏         | 10/500 [02:41<4:13:18, 31.02s/it]

q: 10


  4%|▍         | 20/500 [03:12<32:44,  4.09s/it]  

q: 20


  6%|▌         | 30/500 [03:44<25:38,  3.27s/it]

...

q: 480


 98%|█████████▊| 490/500 [1:20:24<06:29, 38.94s/it]

q: 490


100%|██████████| 500/500 [1:30:51<00:00, 10.90s/it] 

q: 500


In [16]:
train_questions, train_relevances = questions[:300], relevances[:300]
test_questions, test_relevances = questions[300:], relevances[300:]

In [17]:
print(len(train_questions), len(test_questions))

300 200


In [18]:
print(train_questions[:10])

["What is the description of the 'Held' state in the system?", 'What are the faults associated with Station 8 in the CSM Operations Manual?', 'What are the options available for users to modify in the Valve Control Profile Popup?', 'What are the instructions provided in section 14.1?', 'What is the general command sequence for the LDS skid operation?', 'What is the purpose of the vision inspection station in the CSM Operations Manual?', 'What is the function of Station1 in the Vial Filling Connected Smart Manufacturing system?', 'What does the Fill Station 5 Overview Screen provide?', "What is the description of the 'Held' state in the system?", 'What is the purpose of the Idle state in the operation description?']


In [19]:
job_id = db.vectorstore.deep_memory.train(
    queries=train_questions,
    relevance=train_relevances,
)

Starting DeepMemory training job
Your Deep Lake dataset has been successfully created!


Preparing training data for deepmemory:


Creating 300 embeddings in 1 batches of size 300:: 100%|██████████| 1/1 [00:07<00:00,  7.16s/it]


DeepMemory training job started. Job ID: 6570033e5a08341e0ea7095c


In [20]:
db.vectorstore.deep_memory.status("656febcb19cf547449ff8fdf")

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/siddartha10/manufacturing_CSI
--------------------------------------------------------------
|                  656febcb19cf547449ff8fdf                  |
--------------------------------------------------------------
| status                     | completed                     |
--------------------------------------------------------------
| progress                   | eta: 0.9 seconds              |
|                            | recall@10: nan% (+nan%)       |
--------------------------------------------------------------
| results                    | recall@10: nan% (+nan%)       |
--------------------------------------------------------------




In [21]:
recall = db.vectorstore.deep_memory.evaluate(
    queries=test_questions,
    relevance=test_relevances,
)

Embedding queries took 1.74 seconds
---- Evaluating without Deep Memory ---- 
Recall@1:	  53.5%
Recall@3:	  68.5%
Recall@5:	  78.0%
Recall@10:	  88.5%
Recall@50:	  100.0%
Recall@100:	  100.0%
---- Evaluating with Deep Memory ---- 
Recall@1:	  73.5%
Recall@3:	  84.0%
Recall@5:	  89.0%
Recall@10:	  94.5%
Recall@50:	  97.5%
Recall@100:	  98.5%


In [22]:
retriver = db.as_retriever()
retriver.search_kwargs["deep_memory"] = True
retriver.search_kwargs["k"] = 10

query = "what are the different types of stations?"
qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model="gpt-3.5-turbo",temperature=0), chain_type="stuff", retriever=retriver
)
print(qa.run(query))

The different types of stations mentioned in the context are:

1. Station 1 - Load / Unload
2. Station 2 - Dry Fill
3. Station 3 - Wet Fill (Single Nozzle Sequential)
4. Station 4 - Wet Fill (Single Nozzle Mixed)
5. Station 5 - Wet Fill (Individual)
6. Station 6 - Capping
7. Station 7 - Vision Inspection
8. Station 8 - Weigh Check
9. Station 9 - Inspect / Repair


In [23]:
retriver = db.as_retriever()
retriver.search_kwargs["deep_memory"] = True
retriver.search_kwargs["k"] = 10

query = "what is a magne motion?"
qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model="gpt-3.5-turbo",temperature=0), chain_type="stuff", retriever=retriver
)
qa.run(query)

'MagneMotion is a transport system that utilizes linear motor technology and magnetic carriers to transport vials throughout a specific system. It consists of nodes (locating positions) and paths, and the system can stop at process stations or use bypass paths to continue moving. The MagneMotion system is controlled by a node controller and can be viewed and manipulated through MagneMotion screens.'

In [24]:
query = "it is not CSM please refer to it as CSI from next time?"
qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model="gpt-3.5-turbo",temperature=0), chain_type="stuff", retriever=retriver
)
qa.run(query)

'Apologies for the confusion. I will refer to it as CSI from now on.'

In [25]:
query = "How to turn the test bed on?"
qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model="gpt-3.5-turbo",temperature=0), chain_type="stuff", retriever=retriver
)
qa.run(query)

"Based on the provided context, there is no specific information on how to turn on the test bed. It is recommended to refer to the manufacturer's documentation or contact the relevant personnel or support team for instructions on how to turn on the test bed."

In [26]:
query = "Again you are it as CSM no it is CSI?"
qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model="gpt-3.5-turbo",temperature=0), chain_type="stuff", retriever=retriver
)
qa.run(query)

'I apologize for the confusion. The correct acronym is CSI, not CSM.'

In [30]:
query = "What safety measures are in place to protect workers?"
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriver
)
qa.run(query)

"The CSM Operations Manual outlines several safety measures to protect workers. These include the use of safety signs, proper machine operation training, and the importance of maintaining a safe work environment. Additionally, the manual emphasizes the need for safety precautions, such as wearing appropriate clothing, performing safety lockout procedures, and not altering safety guards or devices. The manual also mentions the use of fixed guarding, safety systems, and safety area scanners to prevent contact with hazards. It's important to note that the manual provides detailed instructions for safety, but it's always best to consult with a safety professional for specific workplace safety measures."

In [31]:
query = "Explain me about the Dry Fill station"
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriver
)
qa.run(query)

"I don't have enough information to answer your question."

In [32]:
query = "what is wet Fill station"
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriver
)
qa.run(query)

'A wet fill station is a part of a system used to dispense specified quantities of different fluids into a vial located at the station. There are different types of wet fill stations, such as single nozzle mixed, single nozzle sequential, and wet fill individual. Each type has specific functions and processes for dispensing the fluids into the vial. The system PLC and the fluid delivery system PLC communicate to control the liquid fill parameters and resulting data. The station updates the system’s part tracking and traceability database and then sends the vial on to the next processing station upon completion.'

In [33]:
query = "what is dry fill stations?"
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriver
)
print(qa.run(query))

I don't have that information.


In [34]:
query = "What is Dry cycle mode?"
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriver
)
print(qa.run(query))

Dry Cycle Mode is a mode in which a system emulates an auto sequence without any parts present in the system. It is primarily used for maintenance or troubleshooting purposes. In this mode, the system ignores input sensors and inspection conditions to allow the auto sequence to function. It is similar to the automatic mode, but it does not perform any work and functions in a safe and secure manner.


In [35]:
query = "Explain me about Robot screens?"
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriver
)
print(qa.run(query))

Robot screens are used to view and manipulate the robot functions, I/O, and parameters. They allow for manual control of the robot, including pick, place, and go-to operations, as well as manual control of the robot-controlled I/O functions such as gripper open and close. These screens also display all I/O and interface signals to aid in maintenance and troubleshooting efforts. Each type of robot has its own specific screen, such as the LR-Mate Robot Screen, M1iA Robot Screen, Scara Robot Screen, and CR7 Robot Screen, each with its own set of functions and indicators. These screens also include navigation options to move between different robot screens. Additionally, actions that may cause damage to the equipment or product are interlocked through the PLC program or not allowed to be selected via the HMI.


In [36]:
query = "What does it mean by red flashing lights?"
print(qa.run(query))

Red flashing lights indicate that the system is faulted. This means that there is an issue with the system that needs to be addressed. It's a visual indicator to alert operators that there is a problem that requires attention.


In [37]:
query = "How does the factory test vials for quality?"
print(qa.run(query))

The factory tests vials for quality through a vial inspection station, which inspects the vial for fill level, color accuracy, and weight. The inspection process involves rotating the vial 360 degrees across three different sections, scanning and associating the vial’s 2D bar code data to the scheduled production job, and using a vision system to verify the contents of each vial. Additionally, there is a checkweigher implemented just off the line to measure weight, and a Cognex In-Sight 7802 vision system is used for this application. If the vial meets all process data and inspection criteria, it is considered a "Good Part," and if it fails to meet the criteria, it is considered a "Bad Part."


In [38]:
query = "explain me about CSI test bed?"
output = qa.run(query)
sentences = output.split('. ')
formatted_output = '\n'.join(sentences)
print(formatted_output)

The CSI Test Bed is a system used for testing and assembling components related to vial filling and transport
It involves disassembly and transportation of the test bed from one location to another, as well as the delivery and assembly at the new location
The system includes various components such as the MagneMotion Independent Cart Transport System, vial inspection processes, dry fill stations, and mechanical pressure controls
Additionally, the system is equipped with a safety system, including emergency stop buttons, area scanners, light curtains, and a safety controller to monitor and control the safety of the entire system
The system also involves the use of a robot teach mode for robot adjustment and a dedicated Allen Bradley L8 series GuardLogix programmable safety controller
The test bed is designed to be disassembled into necessary component parts for transportation and then reassembled at the new location.


In [39]:
query = "Explain different types of methods that prevent carts from crashing?"
output = qa.run(query)


In [40]:
print(output)

The VFC (Vial Filling Cell) uses a MagneMotion Independent Cart Transport System to transport vials throughout the system. The system is configured with 17 Nodes (locating positions) on 23 paths. The transport carriers flow through the system, stopping at the required process stations or using bypass paths to continue moving throughout the system. The MagneMotion node controller receives commands from the system’s process PLC and monitors and controls the direct motion of the independent cart trafficking, positioning, and motion profiles. The system is designed to prevent carts from crashing by ensuring that the vehicles do not move within their area or station until all the devices within the area or station are in safe docked positions. Additionally, the system is designed to have vehicle-to-vehicle automatic collision avoidance within the area or stations. This is achieved through a combination of automated control and safety measures to ensure the safe and secure movement of the ca

In [ ]:
query = "How to factorial of a number?"
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriver
)
output = qa.run(query)
sentences = output.split('. ')
formatted_output = '\n'.join(sentences)
print(formatted_output)


I don't know.


In [41]:
query = "Routers explain?"
output = qa.run(query)
sentences = output.split('. ')
formatted_output = '\n'.join(sentences)
print(formatted_output)


I don't have enough information to provide a specific answer to your question
If you could provide more context or specify what you would like to know about routers, I would be happy to help!


In [42]:
query = "what is CSI?"
output = qa.run(query)
sentences = output.split('. ')
formatted_output = '\n'.join(sentences)
print(formatted_output)


CSI stands for Connected Systems Institute, which is a facility at the University of Wisconsin - Milwaukee
It is a platform that integrates advanced manufacturing equipment and techniques to produce vials filled with varying products using a variety of filling methods
The system captures process data for analysis and system optimization and is used for education and research to advance the connected enterprise.


In [43]:
query = "How do we track the vials?"
output = qa.run(query)
sentences = output.split('. ')
formatted_output = '\n'.join(sentences)
print(formatted_output)


The vials are tracked using barcode/QR labels applied to each vial prior to use with the VFC
These labels are printed and applied to the vials to ensure full track-and-trace and serialization capability throughout the system
The vials are tied to a specific MagneMotion puck, and robotic manipulation during vial loading compensates for inconsistent placement of labels on the vial and varying vial orientation within each tray
Additionally, a Teledyne Dalsa Imaging system is used for this application.


In [44]:
query = "explain me about newton's 2nd law?"
prompt = "You are a world class expert for generating questions based on provided context. and you will answer question only based on the given context."
output = qa.run(prompt+query)
sentences = output.split('. ')
formatted_output = '\n'.join(sentences)
print(formatted_output)

I don't have the information to answer that question based on the provided context.


In [45]:
query = "explain me ohms law?"
prompt = "You are a world class expert for generating questions based on provided context. and you will answer question only based on the given context."
output = qa.run(prompt+query)
sentences = output.split('. ')
formatted_output = '\n'.join(sentences)
print(formatted_output)

I'm sorry, but based on the provided context, I cannot answer that question
If you have any questions related to the information provided, feel free to ask and I'll do my best to help!


In [46]:
query = "who is msdhoni?"
prompt = "You are a world class expert for generating questions based on provided context. and you will answer question only based on the given context."
output = qa.run(prompt+query)
sentences = output.split('. ')
formatted_output = '\n'.join(sentences)
print(formatted_output)

I don't know the answer to that question based on the provided context.


In [47]:
def model_answer(query):
    prompt = "You are a world class expert for generating questions based on provided context. and you will answer question only based on the given context."
    output = qa.run(prompt+query)
    sentences = output.split('. ')
    formatted_output = '\n'.join(sentences)
    return formatted_output

In [48]:
print(model_answer("who is ms dhoni?"))

I don't know the answer to that question based on the provided context.


In [51]:
memory = ConversationBufferWindowMemory(
        memory_key='chat_history', return_messages=True)

In [52]:
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriver, memory=memory
)

In [53]:
qa.run("what is CSI?")

'CSI stands for Connected Systems Institute, which is a facility at the University of Wisconsin - Milwaukee. It is a platform that university faculty and students use for education and research to further the advancement of a connected enterprise. The facility utilizes the latest Industry 4.0 connected advanced manufacturing equipment and techniques to produce vials filled with varying products using a variety of filling methods and capturing process data for analysis and system optimization.'

In [54]:
qa.run("")

"I'm sorry, I don't have access to the previous question. How can I assist you today?"

In [57]:
conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type="stuff",
        retriever=retriver,
        memory=memory
    )

In [58]:
conversation_chain.run("what is CSI?")

'CSI stands for Connected Systems Institute.'

In [60]:
conversation_chain.run("can you explain me about it?")

'The purpose of the Connected Systems Institute (CSI) at the University of Wisconsin - Milwaukee is to provide an educational and research platform for faculty and students to further the advancement of a connected enterprise. The institute uses the latest Industry 4.0 connected advanced manufacturing equipment and techniques to produce vials filled with varying products using a variety of filling methods and capturing process data that can be used for data analysis and system optimization.'

In [61]:
conversation_chain.run("what is test bed?")

'A test bed is a setup used for testing and validating the functionality of a system or equipment. In the context provided, the test bed is specifically for the Vial Fill Test Bed, which is being transported and assembled from one location to another. The test bed includes various components and systems, such as robotics, safety controllers, and equipment for filling vials.'

In [62]:
conversation_chain.run("How many are present?")

'The document describes the transport and assembly of a single test bed from the Haskell office to the UWM Campus Library East Wing. Therefore, it appears that there is only one test bed being transported and assembled.'

In [63]:
conversation_chain.run("what are the different types of stations present?")

'The different types of stations present are:\n\n1. Station 1 – Load / Unload\n2. Station 2 – Dry Fill\n3. Station 3 – Wet Fill (Single Nozzle Sequential)\n4. Station 4 – Wet Fill (Single Nozzle Mixed)\n5. Station 5 – Wet Fill (Individual)\n6. Station 6 – Capping\n7. Station 7 – Vision Inspection\n8. Station 8 – Weigh Check\n9. Station 9 – Inspect / Repair\n10. MagneMotion Independent Cart Transport System'

In [66]:
def model_answer_conversation(query):
    prompt = "You are a world class expert for generating questions based on provided context. and you will answer question only based on the given context."
    output = conversation_chain.run(query)
    sentences = output.split('. ')
    formatted_output = '\n'.join(sentences)
    return formatted_output

In [69]:
model_answer_conversation("list the different types of stations?")

'The different types of stations present are:\n\n1\nStation 1 – Load / Unload\n2\nStation 2 – Dry Fill\n3\nStation 3 – Wet Fill (Single Nozzle Sequential)\n4\nStation 4 – Wet Fill (Single Nozzle Mixed)\n5\nStation 5 – Wet Fill (Individual)\n6\nStation 6 – Capping\n7\nStation 7 – Vision Inspection\n8\nStation 8 – Weigh Check\n9\nStation 9 – Inspect / Repair\n10\nMagneMotion Independent Cart Transport System'

In [72]:
model_answer_conversation("Explain the above types in detail in about 100 words?")

'The CSM Operations Manual describes different types of stations, each with its own sequence of operations\nStation 1 is for loading and unloading, Station 2 is for dry fill, Station 3 is for wet fill (single nozzle sequential), Station 4 is for wet fill (single nozzle mixed), Station 5 is for wet fill (individual), Station 6 is for capping, Station 7 is for vision inspection, Station 8 is for weigh check, and Station 9 is for inspect/repair\nEach station has specific steps and functions outlined in the manual, and they are controlled through different modes such as Auto, Manual, and Faulted.'

In [74]:
model_answer_conversation("explain me the modes in the above context?")

"The CSM Operations Manual and the LDS skid system describe several different modes of operation:\n\n1\nControl Power ON/OFF: This mode allows control functions to be turned on and off\nNo motions (auto or manual) can occur if control power is not in the on state.\n\n2\nManual Mode: In this mode, station motions are only allowed through manual interaction\nAny fault condition places the station into this mode of operation.\n\n3\nAuto Mode: The station is ready for automatic operation, but the auto sequence is not started or processing\nNo manual functions can be performed in this mode.\n\n4\nAuto Cycle: The station's auto sequence is processing and repeating under normal conditions.\n\n5\nDry Cycle: This mode emulates the auto cycle without parts or pallets\nIt simulates or ignores input sensors and inspection conditions to allow the auto sequence to function\nIt is primarily used for maintenance or troubleshooting.\n\n6\nFaulted: This mode occurs when the system has experienced a faul